# Dataset Description

In this notebook we'll be performing analysis on **ABALONE** dataset. Abalones are marine snails.
Abalones can be found along coasts of almost every continent. Usually, abalones are consumed as food all around the world, by different cultures.

Anticipating the age of abalone from physical estimations. The age of abalone is dictated by carving the shell through the cone, recoloring it, and tallying the number of rings through a magnifying instrument - an exhausting and tedious task. Different estimations, which are simpler to acquire, are utilized to anticipate the age. Additional data, for example, climate examples and area (thus food accessibility) might be required to take care of the issue.

# Machine Learning Methods Used:

1. Logistic Regression
2. Support Vector Machines
3. K Nearest Neighbors
4. Decision Tree
5. Ensemble Methods (Random Forest, AdaBoost, Gradient Boost)
6. XGBoost

# Importing Libraries and Reading Data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
color = sns.color_palette()
import matplotlib.pyplot as plt

import cufflinks as cf
import plotly.offline as py
color = sns.color_palette()
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import plotly.tools as tls

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

In [ ]:
df = pd.read_csv('abalone.csv')

In [ ]:
df.head(3)

In [ ]:
df.shape

From problem statement and feature discription, let's first compute the target varible of the problem ' Age' and assign it to the dataset. Age = 1.5+Rings

In [ ]:
df['age'] = df['Rings']+1.5
df.drop('Rings', axis = 1, inplace = True)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.var()

# Data Visualization

Distribution of Sex

In [ ]:
df_sex = df['Sex'].value_counts()
print(df_sex.head())
trace = go.Bar(x = df_sex.index[: : -1] ,y = df_sex.values[: : -1], marker = dict(color = 'lightseagreen'))
data = [trace]
layout = go.Layout(height = 400, width = 500, title='Sex Distribution')
fig = go.Figure(data = data, layout= layout)
py.iplot(fig)

Boxplots to look for outliers

In [ ]:
fig, ((ax1, ax2, ax3, ax4), (ax5, ax6, ax7, ax8)) = plt.subplots(nrows= 2, ncols = 4, figsize = (24,10))
sns.boxplot(ax = ax1, y = 'Length', data = df, color = 'green')
sns.boxplot(ax = ax2, y = 'Diameter', data = df, color = 'red')
sns.boxplot(ax = ax3, y = 'Height', data = df, color = 'limegreen')
sns.boxplot(ax = ax4, y = 'Whole weight', data = df, color = 'cyan')
sns.boxplot(ax = ax5, y = 'Shucked weight', data = df, color = 'salmon')
sns.boxplot(ax = ax6, y = 'Viscera weight', data = df, color = 'mediumorchid')
sns.boxplot(ax = ax7, y = 'Shell weight', data = df, color = 'lime')
sns.boxplot(ax = ax8, y = 'age', data = df, color = 'plum')

Feature Distribution using plotly

In [ ]:
df1 = df.copy()
df2 = df.copy()
df3 = df.copy()

df_m = df1[df1['Sex'] == 'M']
df_m.drop('Sex', axis = 1, inplace= True)
df_f = df2[df2['Sex'] == 'F']
df_f.drop('Sex', axis = 1, inplace= True)
df_i = df3[df3['Sex'] == 'I']
df_i.drop('Sex', axis = 1, inplace= True)
df_m.drop(['age'], axis=1, inplace = True)
df_f.drop(['age'], axis=1, inplace = True)
df_i.drop(['age'], axis=1, inplace = True)

df_m = df_m.mean()
df_f = df_f.mean()
df_i = df_i.mean()
trace1 = go.Bar(x = df_m.index[::-1], y = df_m.values[::-1], name = 'M', marker = dict(color = 'cyan'))
trace2 = go.Bar(x = df_f.index[::-1], y = df_f.values[::-1], name = 'F', marker = dict(color = 'violet'))
trace3 = go.Bar(x = df_i.index[::-1], y = df_i.values[::-1], name = 'I', marker = dict(color = 'lightsteelblue'))
data = [trace1, trace2, trace3]
layout = go.Layout(title = 'Feature Distribution', width = 800)
fig = go.Figure(data = data, layout= layout)
py.iplot(fig)

In [ ]:
df4 = df.copy()
df5 = df.copy()
df6 = df.copy()
df_m1 = df4[df4['Sex'] == 'M']
df_m1.drop('Sex', axis = 1, inplace= True)
df_f1 = df5[df5['Sex'] == 'F']
df_f1.drop('Sex', axis = 1, inplace= True)
df_i1 = df6[df6['Sex'] == 'I']
df_i1.drop('Sex', axis = 1, inplace= True)
df_m1.drop(['Length','Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight'], axis=1, inplace = True)
df_f1.drop(['Length','Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight'], axis=1, inplace = True)
df_i1.drop(['Length','Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight'], axis=1, inplace = True)

df_m1 = df_m1.mean()
df_f1 = df_f1.mean()
df_i1 = df_i1.mean()
trace1 = go.Bar(x = df_m1.index[::-1], y = df_m1.values[::-1], name = 'M', marker = dict(color = 'limegreen'))
trace2 = go.Bar(x = df_f1.index[::-1], y = df_f1.values[::-1], name = 'F', marker = dict(color = 'olive'))
trace3 = go.Bar(x = df_i1.index[::-1], y = df_i1.values[::-1], name = 'I', marker = dict(color = 'seagreen'))
data = [trace1, trace2, trace3]
layout = go.Layout(title = 'Feature Distribution', width = 750)
fig = go.Figure(data = data, layout= layout)
py.iplot(fig)

In [ ]:
sns.pairplot(data = df, hue = 'Sex', palette = 'Dark2')

Now we will use seaborn lmplot to look for outliers and removing them.

In [ ]:
sns.set_style('darkgrid')
sns.lmplot(x = 'Length', y = 'age', data = df, hue = 'Sex', palette = 'Set1', scatter_kws={'edgecolor':'white', 'alpha':0.7, 'linewidth':0.5})

In [ ]:
df.drop(df[(df['Length']<0.1) & (df['age'] < 5)].index, inplace=True)
df.drop(df[(df['Length']<0.8) & (df['age'] > 25)].index, inplace=True)
df.drop(df[(df['Length']>=0.8) & (df['age']< 25)].index, inplace=True)

In [ ]:
sns.lmplot(x = 'Diameter', y = 'age', data = df, hue = 'Sex', palette = 'Dark2', scatter_kws={'edgecolor':'white', 'alpha':0.7, 'linewidth':0.5})

In [ ]:
df.drop(df[(df['Diameter']<0.1) & (df['age'] < 5)].index, inplace=True)
df.drop(df[(df['Diameter']<0.6) & (df['age'] > 25)].index, inplace=True)
df.drop(df[(df['Diameter']>=0.6) & (df['age']< 25)].index, inplace=True)

In [ ]:
sns.lmplot(x = 'Height', y = 'age', data = df, hue = 'Sex', palette = 'viridis', scatter_kws={'edgecolor':'white', 'alpha':0.7, 'linewidth':0.5})

In [ ]:
df.drop(df[(df['Height']>0.4) & (df['age'] < 15)].index, inplace=True)
df.drop(df[(df['Height']<0.4) & (df['age'] > 25)].index, inplace=True)

In [ ]:
sns.lmplot(x = 'Whole weight', y = 'age', data = df, hue = 'Sex', palette = 'magma', scatter_kws={'edgecolor':'white', 'alpha':0.7, 'linewidth':0.5})

In [ ]:
df.drop(df[(df['Whole weight']>= 2.5) & (df['age'] < 25)].index, inplace=True)
df.drop(df[(df['Whole weight']<2.5) & (df['age'] > 25)].index, inplace=True)

In [ ]:
sns.lmplot(x = 'Shucked weight', y = 'age', data = df, hue = 'Sex', palette = 'gist_heat', scatter_kws={'edgecolor':'white', 'alpha':0.7, 'linewidth':0.5})

In [ ]:
df.drop(df[(df['Shucked weight']>= 1) & (df['age'] < 20)].index, inplace=True)
df.drop(df[(df['Shucked weight']<1) & (df['age'] > 20)].index, inplace=True)

In [ ]:
sns.lmplot(x = 'Viscera weight', y = 'age', data = df, hue = 'Sex', palette = 'gnuplot', scatter_kws={'edgecolor':'white', 'alpha':0.7, 'linewidth':0.5})

In [ ]:
df.drop(df[(df['Viscera weight']> 0.5) & (df['age'] < 20)].index, inplace=True)
df.drop(df[(df['Viscera weight']<0.5) & (df['age'] > 25)].index, inplace=True)

In [ ]:
sns.lmplot(x = 'Shell weight', y = 'age', data = df, hue = 'Sex', palette = 'twilight_r', scatter_kws={'edgecolor':'white', 'alpha':0.7, 'linewidth':0.5})

In [ ]:
df.drop(df[(df['Shell weight']> 0.6) & (df['age'] < 25)].index, inplace=True)
df.drop(df[(df['Shell weight']<0.8) & (df['age'] > 25)].index, inplace=True)

Now looking for correlation in data using seaborn heatmap.

In [ ]:
df

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Use label encoding for the 'Sex' column
label_encoder = LabelEncoder()
df['Sex'] = label_encoder.fit_transform(df['Sex'])

# Now you can calculate the correlation matrix
correlation_matrix = df.corr()

# Plot the correlation matrix using seaborn or matplotlib
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

# Feature Engineering

We will drop categorical column(Sex).

In [ ]:
df.drop('Sex', axis = 1, inplace = True)
df.head()

In [ ]:
df['age'].value_counts()

In [ ]:
df['age'].mean()

In [ ]:
df_1 = df.copy()

In [ ]:
Age = []
for i in df_1['age']:
    if i > 11.12:
        Age.append('1')
    else:
        Age.append('0')
df_1['Age'] = Age
df_1.drop('age', axis = 1, inplace = True)
df_1.head()

In [ ]:
df_1['Age'].value_counts()

In [ ]:
X = df_1.drop('Age', axis = 1).values
y = df_1['Age'].values

Train Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 123)

Preprocessing

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Time to train model

Logistic Regression

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

print('Classification Report: \n', classification_report(y_test, y_pred))
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
lr_train_acc = lr.score(X_train, y_train)
print('Training Score: ', lr_train_acc)
lr_test_acc = lr.score(X_test, y_test)
print('Testing Score: ', lr_test_acc)

Support Vector Classifiers

In [ ]:
svc = SVC(C = 1, gamma= 1)
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)

print('Classification Report: \n', classification_report(y_test, y_pred))
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
svc_train_acc = svc.score(X_train, y_train) 
print('Training Score: ', svc_train_acc)
svc_test_acc = svc.score(X_test, y_test)
print('Testing Score: ', svc_test_acc)

K Nearest Neighbour Classifier

We will choose number of neighbors.

In [ ]:
error_rate = []
for i in range(1,40):
    knn = KNeighborsClassifier(n_neighbors= i)
    knn.fit(X_train, y_train)
    y_predi = knn.predict(X_test)
    error_rate.append(np.mean(y_test != y_predi))
    
plt.figure(figsize = (10,8))
plt.plot(range(1,40), error_rate, color='blue', linestyle='dashed', marker='o', markerfacecolor='red', markersize=10)

In [ ]:
knn = KNeighborsClassifier(n_neighbors= 31)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

print('Classification Report: \n', classification_report(y_test, y_pred))
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
knn_train_acc = knn.score(X_train, y_train) 
print('Training Score: ', knn_train_acc)
knn_test_acc = knn.score(X_test, y_test)
print('Testing Score: ', knn_test_acc)

Decision Tree Classifier

In [ ]:
dt = DecisionTreeClassifier(max_depth = 5)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)

print('Classification Report: \n', classification_report(y_test, y_pred))
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
dt_train_acc = dt.score(X_train, y_train) 
print('Training Score: ', dt_train_acc)
dt_test_acc = dt.score(X_test, y_test)
print('Testing Score: ', dt_test_acc)

Random Forest Classifier

In [ ]:
rf = RandomForestClassifier(n_estimators= 150, max_depth= 5)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print('Classification Report: \n', classification_report(y_test, y_pred))
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
rf_train_acc = rf.score(X_train, y_train) 
print('Training Score: ', rf_train_acc)
rf_test_acc = rf.score(X_test, y_test)
print('Testing Score: ', rf_test_acc)

AdaBoost Classifier

In [ ]:
adb = AdaBoostClassifier(n_estimators= 100)
adb.fit(X_train, y_train)
y_pred = adb.predict(X_test)

print('Classification Report: \n', classification_report(y_test, y_pred))
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
adb_train_acc = adb.score(X_train, y_train) 
print('Training Score: ', adb_train_acc)
adb_test_acc = adb.score(X_test, y_test)
print('Testing Score: ', adb_test_acc)

Gradient Boosting 

In [ ]:
gdb = GradientBoostingClassifier(n_estimators= 200, max_depth = 2, min_samples_leaf= 2)
gdb.fit(X_train, y_train)
y_pred = gdb.predict(X_test)

print('Classification Report: \n', classification_report(y_test, y_pred))
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
gdb_train_acc = gdb.score(X_train, y_train) 
print('Training Score: ', gdb_train_acc)
gdb_test_acc = gdb.score(X_test, y_test)
print('Testing Score: ', gdb_test_acc)

XGBoost Classifier

In [ ]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.fit_transform(y_test)

In [ ]:
xgb = XGBClassifier(objective = "binary:logistic", n_estimators = 100, max_depth = 3, subsample = 0.8, colsample_bytree = 0.6, learning_rate = 0.1)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)

print('Classification Report: \n', classification_report(y_test, y_pred))
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
xgb_train_acc = xgb.score(X_train, y_train) 
print('Training Score: ', xgb_train_acc)
xgb_test_acc = xgb.score(X_test, y_test)
print('Testing Score: ', xgb_test_acc)

# Plotting Accuracy graph using plotly

In [ ]:
x = ['Logistic Regression','SVC', 'KNN', 'Decision Tree','Random Forest','AdaBoost','Gradient Boosting','XGBoost']
y1 = [lr_train_acc, svc_train_acc, knn_train_acc, dt_train_acc, rf_train_acc, adb_train_acc, gdb_train_acc, xgb_train_acc]
y2 = [lr_test_acc, svc_test_acc, knn_test_acc, dt_test_acc, rf_test_acc, adb_test_acc, gdb_test_acc, xgb_test_acc]

trace1 = go.Bar(x = x, y = y1, name = 'Training Accuracy', marker = dict(color = 'cyan'))
trace2 = go.Bar(x = x, y = y2, name = 'Testing Accuracy', marker = dict(color = 'violet'))
data = [trace1,trace2]
layout = go.Layout(title = 'Accuracy Plot', width = 750)
fig = go.Figure(data = data, layout = layout)
py.iplot(fig)